# Trace Cross Encoder

In [1]:
import torch
from sentence_transformers import CrossEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_id = "cross-encoder/ms-marco-MiniLM-L-12-v2"
model = AutoModelForSequenceClassification.from_pretrained(model_id)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_id)
features = tokenizer([('Query', 'Paragraph1'), ('Query', 'Paragraph2') , ('Query', 'Paragraph3')],  padding=True, truncation=True, return_tensors="pt")

/home/linuxbrew/.linuxbrew/opt/python@3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
features

{'input_ids': tensor([[  101, 23032,   102, 20423,  2487,   102],
        [  101, 23032,   102, 20423,  2475,   102],
        [  101, 23032,   102, 20423,  2509,   102]]), 'token_type_ids': tensor([[0, 0, 0, 1, 1, 1],
        [0, 0, 0, 1, 1, 1],
        [0, 0, 0, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]])}

In [3]:
ex_input = (features["input_ids"], features["token_type_ids"], features["attention_mask"])
traced_model = torch.jit.trace(model, ex_input, strict=False)
torch.jit.save(traced_model, "traced_cross_encoder.pt")

In [4]:
loaded_model = torch.jit.load("traced_cross_encoder.pt")
loaded_model.eval()

RecursiveScriptModule(
  original_name=BertForSequenceClassification
  (bert): RecursiveScriptModule(
    original_name=BertModel
    (embeddings): RecursiveScriptModule(
      original_name=BertEmbeddings
      (word_embeddings): RecursiveScriptModule(original_name=Embedding)
      (position_embeddings): RecursiveScriptModule(original_name=Embedding)
      (token_type_embeddings): RecursiveScriptModule(original_name=Embedding)
      (LayerNorm): RecursiveScriptModule(original_name=LayerNorm)
      (dropout): RecursiveScriptModule(original_name=Dropout)
    )
    (encoder): RecursiveScriptModule(
      original_name=BertEncoder
      (layer): RecursiveScriptModule(
        original_name=ModuleList
        (0): RecursiveScriptModule(
          original_name=BertLayer
          (attention): RecursiveScriptModule(
            original_name=BertAttention
            (self): RecursiveScriptModule(
              original_name=BertSelfAttention
              (query): RecursiveScriptModule(ori

In [5]:
test_sentences = [('How many people live in Berlin?', 'How many people live in Berlin?'), ('Berlin has a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.', 'New York City is famous for the Metropolitan Museum of Art.')]
test_features = tokenizer(test_sentences,  padding=True, truncation=True, return_tensors="pt")
pt_prediction = loaded_model(**test_features)
pt_prediction

{'logits': tensor([[ 4.7880],
         [-9.6987]], grad_fn=<AddmmBackward0>)}

# Compare Output

In [6]:
from sentence_transformers import CrossEncoder
model = CrossEncoder(model_id)
original_embedding = model.predict(test_sentences)
original_embedding

array([ 4.788017, -9.698676], dtype=float32)

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

features = tokenizer(test_sentences,  padding=True, truncation=True, return_tensors="pt")

model.eval()
with torch.no_grad():
    scores = model(**features).logits
    print(scores)

tensor([[ 4.7880],
        [-9.6987]])
